In [1]:
import pandas as pd
import numpy as np
import random
from collections import deque
import json
import talib
import gym
from gym import spaces
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
import warnings
warnings.filterwarnings('ignore')

import plotly.offline as pyo
import plotly.graph_objects as go
from plotly.subplots import make_subplots

pyo.init_notebook_mode(connected=True)

import tensorflow as tf
tf.get_logger().setLevel('ERROR')

class StockTradingEnv(gym.Env):
    """A stock trading environment for OpenAI gym"""
    metadata = {'render.modes': ['cli', 'gui']}

    def preprocess_data(self, df):
        self.df_orig=df.copy(deep=True)
        logScale = pd.DataFrame(np.log(df.Close))
        df["CloseLogDiff"] = logScale - logScale.shift()
        df['Return'] = df.Close.pct_change()
        df['Return_5'] = df.Close.pct_change(5)
        df['Return_10'] = df.Close.pct_change(10)
        df['Return_15'] = df.Close.pct_change(15)
        df['RSI'] = talib.STOCHRSI(df.Close)[1]
        df['MACD'] = talib.MACD(df.Close)[1]
        df['ATR'] = talib.ATR(df.High, df.Low, df.Close)
        slowk, slowd = talib.STOCH(df.High, df.Low, df.Close)
        df['STOCH'] = slowd - slowk
        df['ULTOSC'] = talib.ULTOSC(df.High, df.Low, df.Close)
        df.fillna(method='bfill', inplace=True)
        return df
    
    def __init__(self, df, initial_balance=10000, lookback_window_size=30):
        # Define action space and state size and other custom parameters
        self.df = self.preprocess_data(df)
        self.df = df.dropna().reset_index()
        self.df_total_steps = len(self.df)-1
        self.initial_balance = initial_balance
        self.lookback_window_size = lookback_window_size

        # Action space from 0 to 3, 0 is hold, 1 is buy, 2 is sell
        self.action_space = spaces.Box(low=np.array([0,0]), high=np.array([3,1]), dtype=np.float16)

        # Orders history contains the balance, net_worth, stock_bought, stock_sold, stock_held values for the last lookback_window_size steps
        self.orders_history = deque(maxlen=self.lookback_window_size)
        
        # Market history contains the OHCL values for the last lookback_window_size prices
        self.market_history = deque(maxlen=self.lookback_window_size)

        # State size contains Market+Orders history for the last lookback_window_size steps
        self.state_size = (self.lookback_window_size, 10)

        self.observation_space = spaces.Box(low=0, high=1, shape=(lookback_window_size,15), dtype=np.float16)
        
    # Reset the state of the environment to an initial state
    def reset(self, env_steps_size = 201):
        self.display_reward = 0
        self.balance = self.initial_balance
        self.net_worth = self.initial_balance
        self.prev_net_worth = self.initial_balance
        self.stock_held = 0
        self.stock_sold = 0
        self.stock_bought = 0
        
        if env_steps_size > 0: # used for training dataset
            self.start_step = random.randint(self.lookback_window_size, self.df_total_steps - env_steps_size - 1)
            self.end_step = self.start_step + env_steps_size
        
        else: # used for testing dataset
            self.start_step = self.lookback_window_size
            self.end_step = self.df_total_steps
            
        self.current_step = self.start_step

        for i in reversed(range(self.lookback_window_size)):
            current_step = self.current_step - i
            self.orders_history.append([self.balance, self.net_worth, self.stock_bought, self.stock_sold, self.stock_held])
            self.market_history.append([self.df.loc[current_step, 'CloseLogDiff'],
                                        self.df.loc[current_step, 'Return'],
                                        self.df.loc[current_step, 'Return_5'],
                                        self.df.loc[current_step, 'Return_10'],
                                        self.df.loc[current_step, 'Return_15'],
                                        self.df.loc[current_step, 'RSI'],
                                        self.df.loc[current_step, 'MACD'],
                                        self.df.loc[current_step, 'ATR'],
                                        self.df.loc[current_step, 'STOCH'],
                                        self.df.loc[current_step, 'ULTOSC']
                                        ])

        state = np.concatenate((self.market_history, self.orders_history), axis=1)
        self.trades = []
        return state

    # Get the data points for the given current_step
    def _next_observation(self):
        self.market_history.append([self.df.loc[self.current_step, 'CloseLogDiff'],
                                    self.df.loc[self.current_step, 'Return'],
                                    self.df.loc[self.current_step, 'Return_5'],
                                    self.df.loc[self.current_step, 'Return_10'],
                                    self.df.loc[self.current_step, 'Return_15'],
                                    self.df.loc[self.current_step, 'RSI'],
                                    self.df.loc[self.current_step, 'MACD'],
                                    self.df.loc[self.current_step, 'ATR'],
                                    self.df.loc[self.current_step, 'STOCH'],
                                    self.df.loc[self.current_step, 'ULTOSC'],
                                    ])
        obs = np.concatenate((self.market_history, self.orders_history), axis=1)
        return obs

    # Execute one time step within the environment
    def step(self, action):
        self.stock_bought = 0
        self.stock_sold = 0
        self.current_step += 1

        action_type = action[0]
        action_percent = action[1]
        
        # Set the current price to a random price between open and close
        current_price = random.uniform(
            self.df.loc[self.current_step, 'Open'],
            self.df.loc[self.current_step, 'Close'])
        
        if action_type < 1: # Hold

            self.trades.append({'Date': str(self.df_orig.index[self.current_step]).rstrip("00:00:00"),
                                'Open': self.df_orig.Open[self.current_step],
                                'High': self.df_orig.High[self.current_step],
                                'Low': self.df_orig.Low[self.current_step],
                                'Close': self.df_orig.Close[self.current_step],
                                'Adj Close': self.df_orig['Adj Close'][self.current_step],
                                'Volume': self.df_orig.Volume[self.current_step],
                                'Net Worth': self.balance + self.stock_held * current_price,
                                'Current Price': current_price,
                                'Shares': self.stock_held,
                                'Type': "Hold"})

        elif action_type < 2 and int((self.balance * action_percent) / current_price) >= 1:
            # Buy with % of current balance
            self.stock_bought = int((self.balance * action_percent) / current_price)
            self.balance -= self.stock_bought * current_price
            self.stock_held += self.stock_bought
                        
            self.trades.append({'Date': str(self.df_orig.index[self.current_step]).rstrip("00:00:00"),
                                'Open': self.df_orig.Open[self.current_step],
                                'High': self.df_orig.High[self.current_step],
                                'Low': self.df_orig.Low[self.current_step],
                                'Close': self.df_orig.Close[self.current_step],
                                'Adj Close': self.df_orig['Adj Close'][self.current_step],
                                'Volume': self.df_orig.Volume[self.current_step],
                                'Net Worth': self.balance + self.stock_held * current_price,
                                'Current Price': current_price,
                                'Shares': self.stock_bought,
                                'Type': "Buy"})

        elif action_type < 3 and int(self.stock_held * action_percent) >= 1:
            # Sell % of current stock held
            self.stock_sold = int(self.stock_held * action_percent)
            self.balance += self.stock_sold * current_price
            self.stock_held -= self.stock_sold

            self.trades.append({'Date': str(self.df_orig.index[self.current_step]).rstrip("00:00:00"),
                                'Open': self.df_orig.Open[self.current_step],
                                'High': self.df_orig.High[self.current_step],
                                'Low': self.df_orig.Low[self.current_step],
                                'Close': self.df_orig.Close[self.current_step],
                                'Adj Close': self.df_orig['Adj Close'][self.current_step],
                                'Volume': self.df_orig.Volume[self.current_step],
                                'Net Worth': self.balance + self.stock_held * current_price,
                                'Current Price': current_price,
                                'Shares': self.stock_sold,
                                'Type': "Sell"})

            
        self.prev_net_worth = self.net_worth
        self.net_worth = self.balance + self.stock_held * current_price

        self.orders_history.append([self.balance, self.net_worth, self.stock_bought, self.stock_sold, self.stock_held])

        # Calculate reward
        reward = int((self.net_worth - self.initial_balance))
        self.display_reward = reward
        
        if self.net_worth <= self.initial_balance/2:
            done = True
        elif self.current_step > self.end_step:
            done = True
        else:
            done = False


        obs = self._next_observation()

        info = {'Step': self.current_step, 'Trades' : self.trades}
    
        return obs, reward, done, info

    # render environment
    def render(self, mode='cli'):
        if mode == 'cli':
            self.profit = self.net_worth - self.initial_balance
            print('Step:%5d, Date: %10s, Reward:%4d, Stock [Held: %3d, Bought: %3d, Sold :%3d], NetWorth: %5.2f, Profit: %5.2f' % 
                      (self.current_step, str(self.df_orig.index[self.current_step]).rstrip("00:00:00"),
                       self.display_reward, self.stock_held, self.stock_bought, self.stock_sold, self.net_worth, self.profit))
        if mode == 'gui':
            pass


In [2]:
# Change the working directory to the currect path (of .ipynb and csv)
import os
import sys
import math 
os.chdir(sys.path[0])  
df = pd.read_csv('./KAJARIACER.csv')
df = df.sort_values('Date')

In [3]:
df['Date'] = pd.to_datetime(df['Date']) # Convert "Date" to datetime object.
df.sort_values('Date', ascending=True, inplace=True) # Sort the "Date" Column by the values.
df.set_index('Date', inplace=True) # Set the DataFrame index using "Date" Column.

In [4]:
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,26.049999,28.200001,26.049999,27.000000,23.964306,877408
2010-01-05,27.125000,28.200001,26.875000,27.125000,24.075254,508398
2010-01-06,27.500000,28.400000,26.625000,26.825001,23.808989,402216
2010-01-07,27.250000,27.549999,26.850000,27.100000,24.053064,478980
2010-01-08,27.250000,28.025000,26.525000,26.700001,23.698040,366642


In [5]:
def Random_games(env):
    env.reset()
    for _ in range(10):
        env.render()
        action=env.action_space.sample()
#         print(action)
        env.step(action) # take a random action
    env.close()

In [6]:
import talib
import pandas as pd


diff = lambda x, y: x - y
abs_diff = lambda x, y: abs(x - y)


indicators = [
    ('RSI', talib.STOCHRSI, ['Close'])    
]


def add_indicators(df) -> pd.DataFrame:
    for name, f, arg_names in indicators:
        wrapper = lambda func, args: func(*args)
        args = [df[arg_name] for arg_name in arg_names]
        df[name] = wrapper(f, args)
    df.fillna(method='bfill', inplace=True)
    return df

In [7]:
import gym
import json
import datetime as dt
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import PPO2
# from StockTradingEnv import StockTradingEnv

In [8]:
# Train data = 70%, Validation data = 20%, Test data = 10%  
train_range = math.floor(len(df.index)*.7)
val_range = math.floor(len(df.index)*.2)
test_range = math.floor(len(df.index)*.1)

# Train DataFrame
train_df = df[:train_range].sort_index()

# Validation DataFrame
val_df = df[train_range:train_range + val_range].sort_index()

# Test DataFrame
test_df = df[(train_range + val_range):(train_range + val_range + test_range)].sort_index()

In [9]:
import optuna
from optuna import Trial, visualization

from optuna.samplers import TPESampler

In [10]:
def optimize_ppo2_mlp(trial):
    return {
        'n_steps': int(trial.suggest_loguniform('n_steps', 16, 2048)),
        'gamma': trial.suggest_loguniform('gamma', 0.9, 0.9999),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1.),
        'ent_coef': trial.suggest_loguniform('ent_coef', 1e-8, 1e-1),
        'cliprange': trial.suggest_uniform('cliprange', 0.1, 0.4),
        'noptepochs': int(trial.suggest_loguniform('noptepochs', 1, 48)),
        'lam': trial.suggest_uniform('lam', 0.8, 1.)
    }

In [11]:
def objective_ppo2_mlp(trial):
    agent_params = optimize_ppo2_mlp(trial)
    
    train_env = DummyVecEnv([lambda: StockTradingEnv(train_df)])
    model = PPO2(MlpPolicy, train_env, nminibatches=1, **agent_params,  tensorboard_log="./tensorboard/PPO2_MLP")
    
    model.learn(total_timesteps=len(train_df))
    
    rewards, done = [], False

    val_env = DummyVecEnv([lambda: StockTradingEnv(val_df)])
    obs = val_env.reset()
    
    for i in range(len(val_df)):
        action, _states = model.predict(obs, deterministic=False)
        obs, reward, done, info = val_env.step(action)
        rewards.append(reward)
        
    return -np.mean(rewards)

In [12]:
optuna.delete_study(study_name='PPO2_MLP_optimize_profit', storage='sqlite:///params.db')
study_PPO2_MLP = optuna.create_study(study_name='PPO2_MLP_optimize_profit', storage='sqlite:///params.db', 
                            load_if_exists=True, sampler=TPESampler())
study_PPO2_MLP.optimize(objective_ppo2_mlp, n_trials=40, n_jobs=4, show_progress_bar = True)

[I 2022-04-10 10:58:02,243] A new study created in RDB with name: PPO2_MLP_optimize_profit
[I 2022-04-10 10:58:25,320] Trial 1 finished with value: 1092.2906494140625 and parameters: {'n_steps': 799.3245448629137, 'gamma': 0.9516877534778488, 'learning_rate': 0.006247350420059821, 'ent_coef': 7.687110416905082e-08, 'cliprange': 0.28345253692709843, 'noptepochs': 4.439588881911275, 'lam': 0.9961116007730965}. Best is trial 1 with value: 1092.2906494140625.
[I 2022-04-10 10:58:27,480] Trial 0 finished with value: -0.0 and parameters: {'n_steps': 124.85510579857383, 'gamma': 0.9699742264854411, 'learning_rate': 0.13885105287002955, 'ent_coef': 4.6291274861937185e-07, 'cliprange': 0.10535447774891055, 'noptepochs': 9.012785503634982, 'lam': 0.9062333248933477}. Best is trial 0 with value: 0.0.
[I 2022-04-10 10:58:27,520] Trial 2 finished with value: 39.05487823486328 and parameters: {'n_steps': 66.16806161773671, 'gamma': 0.9491785712261684, 'learning_rate': 2.0491160416283963e-05, 'ent_co

[I 2022-04-10 11:00:46,397] Trial 22 finished with value: -0.0 and parameters: {'n_steps': 159.370377962833, 'gamma': 0.997184252746593, 'learning_rate': 0.7431909090780872, 'ent_coef': 0.0001897996710314604, 'cliprange': 0.22033414142144098, 'noptepochs': 2.898108954862357, 'lam': 0.8089452583039596}. Best is trial 6 with value: -667.5751953125.
[I 2022-04-10 11:00:48,954] Trial 23 finished with value: -293.4776306152344 and parameters: {'n_steps': 170.11930089691182, 'gamma': 0.9960576847195753, 'learning_rate': 0.016595526189047437, 'ent_coef': 1.768276278791295e-08, 'cliprange': 0.18604216982044391, 'noptepochs': 2.2852946464154056, 'lam': 0.8034253102552769}. Best is trial 6 with value: -667.5751953125.
[I 2022-04-10 11:00:52,543] Trial 24 finished with value: 338.7804870605469 and parameters: {'n_steps': 161.95711168966386, 'gamma': 0.9009032342048726, 'learning_rate': 0.013254620663306209, 'ent_coef': 0.0041618677488005264, 'cliprange': 0.3142665878721625, 'noptepochs': 26.38343

In [13]:
study_PPO2_MLP.best_params

{'cliprange': 0.1922357730165789,
 'ent_coef': 1.2629218417257388e-08,
 'gamma': 0.9703336836321487,
 'lam': 0.9692877319759365,
 'learning_rate': 0.012366075775935122,
 'n_steps': 1877.7335702866903,
 'noptepochs': 1.765467533578753}

In [14]:
optuna.visualization.plot_optimization_history(study_PPO2_MLP)

In [15]:
optuna.visualization.plot_slice(study_PPO2_MLP)

In [16]:
study_PPO2_MLP = optuna.load_study(study_name='PPO2_MLP_optimize_profit', storage='sqlite:///params.db')
params = study_PPO2_MLP.best_trial.params

train_env = DummyVecEnv([lambda: StockTradingEnv(train_df)])

model_params = {
    'n_steps': int(params['n_steps']),
    'gamma': params['gamma'],
    'learning_rate': params['learning_rate'],
    'ent_coef': params['ent_coef'],
    'cliprange': params['cliprange'],
    'noptepochs': int(params['noptepochs']),
    'lam': params['lam']
}

model = PPO2(MlpPolicy, train_env, nminibatches=1, **model_params)

In [17]:
model.learn(total_timesteps=100000)

In [18]:
test_env = DummyVecEnv([lambda: StockTradingEnv(test_df)])

In [19]:
obs = test_env.reset()

for i in range(200):
    action, _states = model.predict(obs, deterministic=False)
    obs, reward, done, info = test_env.step(action)
    test_env.render(mode='cli')
    if done:
        obs = test_env.reset()
        break

Step:   38, Date: 2019-02-18 , Reward:   0, Stock [Held:   0, Bought:   0, Sold :  0], NetWorth: 10000.00, Profit:  0.00
Step:   39, Date: 2019-02-19 , Reward:   0, Stock [Held:   0, Bought:   0, Sold :  0], NetWorth: 10000.00, Profit:  0.00
Step:   40, Date: 2019-02-20 , Reward:   0, Stock [Held:   0, Bought:   0, Sold :  0], NetWorth: 10000.00, Profit:  0.00
Step:   41, Date: 2019-02-21 , Reward:   0, Stock [Held:   0, Bought:   0, Sold :  0], NetWorth: 10000.00, Profit:  0.00
Step:   42, Date: 2019-02-22 , Reward:   0, Stock [Held:   0, Bought:   0, Sold :  0], NetWorth: 10000.00, Profit:  0.00
Step:   43, Date: 2019-02-25 , Reward:   0, Stock [Held:   0, Bought:   0, Sold :  0], NetWorth: 10000.00, Profit:  0.00
Step:   44, Date: 2019-02-26 , Reward:   0, Stock [Held:   0, Bought:   0, Sold :  0], NetWorth: 10000.00, Profit:  0.00
Step:   45, Date: 2019-02-27 , Reward:   0, Stock [Held:   0, Bought:   0, Sold :  0], NetWorth: 10000.00, Profit:  0.00
Step:   46, Date: 2019-02-28 , R

Step:  142, Date: 2019-07-24 , Reward: 254, Stock [Held:   8, Bought:   0, Sold :  0], NetWorth: 10254.08, Profit: 254.08
Step:  143, Date: 2019-07-25 , Reward:  -8, Stock [Held:   8, Bought:   0, Sold :  0], NetWorth: 9991.78, Profit: -8.22
Step:  144, Date: 2019-07-26 , Reward:  44, Stock [Held:   8, Bought:   0, Sold :  0], NetWorth: 10044.98, Profit: 44.98
Step:  145, Date: 2019-07-29 , Reward:  41, Stock [Held:   8, Bought:   0, Sold :  0], NetWorth: 10041.89, Profit: 41.89
Step:  146, Date: 2019-07-30 , Reward: -44, Stock [Held:   8, Bought:   0, Sold :  0], NetWorth: 9955.36, Profit: -44.64
Step:  147, Date: 2019-07-31 , Reward: -57, Stock [Held:   8, Bought:   0, Sold :  0], NetWorth: 9942.33, Profit: -57.67
Step:  148, Date: 2019-08-01 , Reward: -61, Stock [Held:   8, Bought:   0, Sold :  0], NetWorth: 9938.72, Profit: -61.28
Step:  149, Date: 2019-08-02 , Reward: -61, Stock [Held:   8, Bought:   0, Sold :  0], NetWorth: 9938.05, Profit: -61.95
Step:  150, Date: 2019-08-05 , R

Step:  233, Date: 2019-12-11 , Reward:1303, Stock [Held:  19, Bought:   0, Sold :  0], NetWorth: 11303.30, Profit: 1303.30
Step:  234, Date: 2019-12-12 , Reward:1394, Stock [Held:  19, Bought:   0, Sold :  0], NetWorth: 11395.00, Profit: 1395.00
Step:  235, Date: 2019-12-13 , Reward:1547, Stock [Held:  19, Bought:   0, Sold :  0], NetWorth: 11547.11, Profit: 1547.11
Step:  236, Date: 2019-12-16 , Reward:1460, Stock [Held:  19, Bought:   0, Sold :  0], NetWorth: 11460.73, Profit: 1460.73
Step:  237, Date: 2019-12-17 , Reward:1182, Stock [Held:  19, Bought:   0, Sold :  0], NetWorth: 11182.02, Profit: 1182.02


In [20]:
trades = pd.DataFrame(info[0].get('Trades'))
trades['Date'] = pd.to_datetime(trades['Date']) # Convert "Date" to datetime object.
df.sort_values('Date', ascending=True, inplace=True) # Sort the "Date" Column by the values.

In [21]:
def display_trades(df):
    
    # Convert the column names from mixcase to lower case
    df.columns = map(str.lower, df.columns)
    
    # Fix date
    if type(df.index) != pd.core.indexes.datetimes.DatetimeIndex:
        df['date'] = pd.to_datetime(df['date'])
        df.set_index('date',inplace=True) # Set the Date column as the index
        df = df.sort_values(['date'])
    
     # Copy the dataframe
    tempdf=df.copy(deep=True)
    
    # Calculate profit
    
    profit = round(tempdf.iloc[-1]['net worth'] - tempdf.iloc[1]['net worth'], 2)
    profit_prcnt = round(((tempdf.iloc[-1]['net worth'] - tempdf.iloc[1]['net worth']) / tempdf.iloc[1]['net worth']) * 100 , 2)
    
    trade_buy = tempdf['type'][tempdf['type']=='Buy'].count()
    trade_sell = tempdf['type'][tempdf['type']=='Sell'].count()
    trade_hold = tempdf['type'][tempdf['type']=='Hold'].count()
    
    title_1 = 'RL Algorithm: PPO2_MLP' + ' - ' + 'Trade Count ' + '[' + 'Buy: ' + str(trade_buy) + ' | ' + 'Sell: ' + str(trade_sell) + ' | ' + 'Hold: ' + str(trade_hold) + ']'
    title_2 = 'Initial Investment (INR): ' + str(round(tempdf.iloc[1]['net worth'],2)) + ' | ' +  'Net Worth (INR): ' + str(round(tempdf.iloc[-1]['net worth'],2)) + ' | ' + 'Profit (INR): ' + str(profit) + ' | ' + 'Profit (%): ' + str(profit_prcnt)+ '%'
    
    
    # Buy sell marker
    d=10
    tempdf[['marker', 'symbol', 'color']] = ''
    for i in range(0, len(tempdf)):
        if tempdf['type'].iloc[i] == 'Buy':
            tempdf['marker'].iloc[i] = tempdf["low"].iloc[i]-d
            tempdf['symbol'].iloc[i] = 'triangle-up'
            tempdf['color'].iloc[i] = 'green'
            
        elif tempdf['type'].iloc[i] == 'Sell':
            tempdf['marker'].iloc[i] = tempdf["high"].iloc[i]+d
            tempdf['symbol'].iloc[i] = 'triangle-down'
            tempdf['color'].iloc[i] = 'red'
        
        else:
            tempdf['marker'].iloc[i] = tempdf["low"].iloc[i]-d
            tempdf['symbol'].iloc[i] = 'cross-thin'
            tempdf['color'].iloc[i] = 'blue' 

    
    # Draw plot
    fig = make_subplots(rows=2, cols=1, 
                    shared_xaxes=True, 
                    row_width=[.25,.75], 
                    vertical_spacing=0.05,
                   subplot_titles=(title_1, title_2))

   
    fig.add_trace(go.Scatter(x=tempdf.index,
                             y=tempdf.marker,
                             mode='markers',
                             name='Buy/Sell/Hold',
                             marker=dict(size=15,symbol=tempdf["symbol"],color=tempdf["color"])),
                   row=1, col=1)
                  
    fig.add_trace(go.Candlestick(x=tempdf.index, 
                                 open=tempdf['open'], 
                                 high=tempdf['high'], 
                                 low=tempdf['low'], 
                                 close=tempdf['close'],
                                 name='OHLC'),
                  row=1, col=1)
   
                  
    fig.add_trace(go.Scatter(x=tempdf.index,
                             y=tempdf['net worth'],
                             mode='lines',
                             name='Net Worth (INR)'),
                  row=2, col=1)
    
    fig.add_hline(y=10000,
                  name='Initial Balance',
                  line=dict(color='firebrick', width=1, dash='dot'),
                  row=2, col=1)

    fig.update_layout(##title='Historical stock price chart',
                      xaxis_tickfont_size=12,
                      yaxis=dict(title='Price (INR/share)',
                                 titlefont_size=14,
                                 tickfont_size=10),
                      xaxis_rangeslider_visible=False,
                      width=1000,height=600, margin=dict(l=50, r=50, b=100, t=100, pad=4),
                      template='none')

    fig.show()

In [22]:
display_trades(trades)

In [23]:
# # %load_ext tensorboard
# %reload_ext tensorboard
# %tensorboard \
#     --host localhost \
#     --port 8080\
#     --logdir ".\\tensorboard\\PPO2_MLP"

To view: http://localhost:8080/ <br>
To kill: `taskkill /IM "tensorboard.exe" /F`